# Swap LRNA
Agent $\alpha$ swaps asset $i$ with the protocol for LRNA.

## Constraints

The swap will keep the swap invariant for asset $i$, $R_iQ_i$, constant.

## Requirements

#### Case 1: Asset $i$ sold
$$
\Delta r^\alpha \leq r_i^\alpha
$$
#### Case 2: HDX sold
$$
\Delta q^\alpha \leq q^\alpha
$$

## Updating AMM state

### Case 1: LRNA sold, $\Delta q^\alpha > 0$ specified

If $\Delta q^\alpha > q^\alpha$, the user does not have enough LRNA to sell, and the transaction must fail.

$$
\begin{align}
\Delta Q_i &= \Delta q^\alpha\\
\Delta R_i &= R_i\frac{- \Delta Q_i}{Q_i + \Delta Q_i}(1 - f_A)\\
\Delta L &= -\Delta Q_i\left(1 + (1 - f_A)\frac{Q_i}{Q_i + \Delta Q_i}\right)\\
\Delta r_i^\alpha &= - \Delta R_i\\
\end{align}
$$

### Case 2: Asset $i$ bought, $\Delta r_i^\alpha > 0$ specified

$$
\begin{align}
\Delta R_i &= -\Delta r_i^\alpha\\
\Delta Q_i &= Q_i\frac{-\Delta R_i}{R_i(1 - f_A) + \Delta R_i}\\
\Delta L &= -\Delta Q_i\left(1 + (1 - f_A)\frac{Q_i}{Q_i + \Delta Q_i}\right)\\
\Delta q^\alpha &= - \Delta Q_i\\
\end{align}
$$

If $\Delta q^\alpha > q^\alpha$, the user does not have enough LRNA to sell, and the transaction must fail.

## Checking the algebra

In [1]:
from sympy.solvers import solve
from sympy import Symbol
from sympy import Max


### Case 1

We want
- $Q_i R_i = (Q_i + \Delta Q_i)(R_i + \Delta R_i)$ when fees are zero
- $\frac{Q_i}{R_i} = \frac{Q_i + \Delta Q_i + \Delta L}{R_i + \Delta R_i}$, long-term price mean reversion

In [2]:
R = Symbol('R')
Q = Symbol('Q')
L = Symbol('L')
fA = Symbol('fA')

dq = Symbol('dq')
dQ = dq
dR = R * (-dQ/(Q + dQ))*(1 - fA)
dL = -dQ*(1 + (1 - fA)*Q/(Q + dQ))
dr = -dR

print("Case 1: price mean-reversion")
lhs = Q/R
rhs = (Q + dQ + dL)/(R + dR)
print(lhs.equals(rhs))

print("Case 1, feeless: swap invariant")
lhs = Q * R
rhs_fee = (Q + dQ)*(R + dR)
rhs_feeless = rhs_fee.subs(fA, 0)
print(lhs.equals(rhs_feeless))

Case 1: price mean-reversion
True
Case 1, feeless: swap invariant
True


### Case 2

As in case 2, we want
- $Q_i R_i = (Q_i + \Delta Q_i)(R_i + \Delta R_i)$ when fees are zero
- $\frac{Q_i}{R_i} = \frac{Q_i + \Delta Q_i + \Delta L}{R_i + \Delta R_i}$, long-term price mean reversion

In [3]:
R = Symbol('R')
Q = Symbol('Q')
L = Symbol('L')
fA = Symbol('fA')

dr = Symbol('dr')
dR = -dr
dQ = Q * -dR/(R*(1 - fA) + dR)
dL = -dQ*(1 + (1 - fA)*Q/(Q + dQ))
dq = -dQ

print("Case 2: price mean-reversion")
lhs = Q/R
rhs = (Q + dQ + dL)/(R + dR)
print(lhs.equals(rhs))

print("Case 2, feeless: swap invariant")
lhs = Q * R
rhs_fee = (Q + dQ)*(R + dR)
rhs_feeless = rhs_fee.subs(fA, 0)
print(lhs.equals(rhs_feeless))

Case 2: price mean-reversion
True
Case 2, feeless: swap invariant
True


We furthermore wish to check that given the amount of LRNA coming into the pool ($\Delta Q_i$), the fee is $f_Q \Delta R_i^*$ where $\Delta R_i^*$ is the amount of asset $i$ sold for $\Delta Q_i$ if fees were zero.
In other words,
- $\Delta R_i = R_i\frac{-\Delta Q_i}{Q + \Delta Q_i}(1 - f_A)$

In [4]:
print("Case 2: Fee calculation")
lhs = dR
rhs = R * (-dQ/(Q + dQ))*(1 - fA)
print(lhs.equals(rhs))

Case 2: Fee calculation
True
